In [1]:
import functions_kinegeo
from functions_kinegeo import *

R[write to console]: data.table 1.13.6 using 12 threads (see ?getDTthreads).  Latest news: r-datatable.com

R[write to console]: Lade nötiges Paket: bit

R[write to console]: 
Attache Paket: ‘bit’


R[write to console]: The following object is masked from ‘package:data.table’:

    setattr


R[write to console]: The following object is masked from ‘package:base’:

    xor


R[write to console]: Attaching package bit64

R[write to console]: package:bit64 (c) 2011-2017 Jens Oehlschlaegel

R[write to console]: creators: integer64 runif64 seq :

R[write to console]: coercion: as.integer64 as.vector as.logical as.integer as.double as.character as.bitstring

R[write to console]: logical operator: ! & | xor != == < <= >= >

R[write to console]: arithmetic operator: + - * / %/% %% ^

R[write to console]: math: sign abs sqrt log log2 log10

R[write to console]: math: floor ceiling trunc round

R[write to console]: querying: is.integer64 is.vector [is.atomic} [length] format print str

R[write t

In [8]:
# test whether prior works
kinegeo_prior_stats(p=6200,r=2000)

array([-63.42389743,  53.87836673,  -1.05965793,  37.41708628,
        -0.24596455])

In [3]:
source_id = 3490289711213205632 #random star from Gaia DR3 of HP 6200
model = 'GGD'

# Settings for metropolis algorithm sampling the distance posterior; rInit, rStep calculated in code

Nsamp = int(5e2) #N_g 
Nburnin = int(5e1)

# Number of velocity samples drawn for reach of the distance samples 

n = 10

#for quantiles

HDIprob = 0.6827
probs=np.array([0.5,(1-HDIprob)/2,(1+HDIprob)/2])

with open('prior_summary.csv', newline='') as prior_summary:
    reader_prior_summary = csv.reader(prior_summary)
    rows_prior_summary = []
    for row in reader_prior_summary:
        rows_prior_summary.append(row)


    job = Gaia.launch_job("select top 10 "
                                            "source_id, parallax, parallax_error,phot_g_mean_mag, nu_eff_used_in_astrometry, pseudocolour, ecl_lat, astrometric_params_solved,bp_rp,pmra,pmra_error,pmdec, pmdec_error,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr "
                                            "from gaiadr3.gaia_source "
                                            "where source_id={}".format(int(source_id)))
    r = job.get_results()
    
    w= float(r['parallax'])
    wsd = float(r['parallax_error'])   
    phot_g_mean_mag = float(r['phot_g_mean_mag'])
    nu_eff_used_in_astrometry = float(r['nu_eff_used_in_astrometry'])
    pseudocolour = float(r['pseudocolour'])
    ecl_lat = float(r['ecl_lat'])
    astrometric_params_solved = float(r['astrometric_params_solved'])
    bp_rp = float(r['bp_rp'])
    
    mu_ra = float(r['pmra'])
    mu_dec = float(r['pmdec'])
    sd_mu_ra = float(r['pmra_error'])
    sd_mu_dec = float(r['pmdec_error'])
    corr_w_mu_ra = float(r['parallax_pmra_corr'])
    corr_w_mu_dec = float(r['parallax_pmdec_corr'])
    corr_mu_ra_dec = float(r['pmra_pmdec_corr'])
    
    source_id = int(r['source_id'])
    hp = math.floor(source_id / (2**(35)*4**(12-5)) )
    
    print('Gaia DR3 source_id:',source_id)
    print('HEALpixel level 5:',hp)
    print('Model of distance prior: ', model)
    print('')
    print('\033[1m' + 'Distance estimation:' + '\033[0m')
    
    #Zeropoint:
                
    if astrometric_params_solved == 31 or astrometric_params_solved == 95:
        
        if  phot_g_mean_mag == np.nan:
            wzp = -0.017
                        
        else:
            wzp = zpt.get_zpt(phot_g_mean_mag, nu_eff_used_in_astrometry, pseudocolour, ecl_lat, astrometric_params_solved)
                    
        #Correct for Zeropoint:
        w = w - wzp
                    
    if model == 'EDSD' or model == 'Photogeometric':
        rlen = float(rows_prior_summary[hp+1][10])  #EDSDrlen in csv-file
    if model == 'GGD':  
        rlen = float(rows_prior_summary[hp+1][5]) # GGDrlen in csv-file
            
    rlen_EDSD = float(rows_prior_summary[hp+1][10])      
    alpha = float(rows_prior_summary[hp+1][6])
    beta = float(rows_prior_summary[hp+1][7])
    glon = float(rows_prior_summary[hp+1][1])
    glat = float(rows_prior_summary[hp+1][2])
    
    rInit = float(mode_post3(w=1e-3*w,wsd=1e-3*wsd,rlen = rlen_EDSD,retall = False)) 
    rStep = 0.75*rInit*min(1/3, abs(wsd/w))
    
    
    # Convert inputs        
    
    w = 1e-3*w
    wsd = 1e-3*wsd
    if astrometric_params_solved == 31 or astrometric_params_solved == 95:
        wzp = 1e-3*wzp
        
    if model == 'EDSD':
        failMessage = np.nan
        rQuant0 = quantile_distpost3(w=w,wsd=wsd,rlen=rlen,rInit=rInit,rStep=rStep,Nburnin=Nburnin,Nsamp=Nsamp,probs=np.array([0.5,0.159,0.841])) #(1-HDIprob)/2,(1+HDIprob)/2
        rQuant = rQuant0[0] #quantiles of distance 
        rSamp = rQuant0[1] #distance samples
        
    if model == 'GGD':
        failMessage = np.nan
        rQuant0 = quantile_distpost4(w=w,wsd=wsd,rlen=rlen,alpha=alpha,beta=beta,rStep=rStep,rInit=rInit,Nburnin=Nburnin,Nsamp=Nsamp,probs=np.array([0.5,0.159,0.841]))
        rQuant = rQuant0[0] #quantiles of distance 
        rSamp = rQuant0[1] #distance samples
        
    if model == 'Photogeometric':
        
        failMessage = np.nan
        rQuant0 = quantile_distpost5(w=w,wsd=wsd,hp=hp,phot_g_mean_mag=phot_g_mean_mag,bp_rp=bp_rp,pseudocolour=pseudocolour,rInit=rInit,Nsamp=Nsamp,Nburnin=Nburnin,probs=np.array([0.5,0.159,0.841]))
        rQuant = rQuant0[0] #quantiles of distance 
        rSamp = rQuant0[1] #distance samples
        
        # QGmodel (only for photogeometric output statistics):
        if rQuant['code'] == 1:
            rFlag = rQuant0[2]
            
    # if distance estimation works: get array with quantiles, otherwise array with nans     
    
    if rQuant['code'] == 1:
        rRes = np.array([rQuant['val'][0],rQuant['val'][1],rQuant['val'][2],2])
        
    else:
        
        failMessage = rQuant['message']
        
    if failMessage == rQuant['message']:
        
        rRes = np.empty((6,))
        rRes[:] = np.nan
        rRes = np.array([rRes[0],rRes[1],rRes[2],rRes[3],rRes[4],0])    
        rSamp = np.nan
        
    if type(failMessage) == str :
        
        print('')
        print(f"\033[1;31m {failMessage} \033[0m")
        print ('') 
        
    else:    
        #print parameters
        print('')
        if astrometric_params_solved == 31 or astrometric_params_solved == 95:
            print('w[mas]',1e3*(w+wzp))
            print('Zeropoint wzp [mas]', 1e3*wzp)
            print('Zeropointcorrected parallax w[mas]',1e3*w)
        else:
            print('w[mas]',1e3*w)
            print('no zeropoint correction, because astrometric_params_solved is not 31 or 95 ')
        print('wsd/w', wsd/w)
    
        if model == 'GGD' or model == 'Photogeometric':
            print('alpha',alpha)
            print('beta',beta)
        
        if model == 'Photogeometric':
            print('G',phot_g_mean_mag)
            print('colour bp-rp',bp_rp)
            
        print('rlen [pc]:', rlen)        
        print('')
        print(f'glon [deg] for HEALpixel {hp}:',glon)
        print(f'glat [deg] for HEALpixel {hp}:',glat)
        print('')
        print('Estimated distances:')
        print('')
        print('r_est [pc] (quantile 0.5):',rRes[0])
        print('r_lo_est [pc] (quantile 0.159):', rRes[1])
        print('r_hi_est [pc] (quantile 0.841):', rRes[2])
        print('')
        
        #print('result_flag', rRes[3]) 
        print('')
        print('MCMC initialization [pc]:', rInit)
        print('MCMC stepsize [pc]:',rStep)
        #print('MCMC Number of iterations[pc]:',Nsamp+Nburnin)
        print('MCMC number of burn-in samples:',Nburnin)
        print('MCMC number of retained iterations:',Nsamp)
        print('')
        
        if model == 'Photogeometric' and rQuant['code'] == 1:
            print('QGmodel:',rFlag[3])
        print('')
    
    
    # kinegeometric velocity estimation:
    
    # Convert inputs back to mas and kpc  
    
    w = 1e3*w
    wsd = 1e3*wsd
    rSamp = 1e-3*rSamp
    
    vRes = quantile_velpost(rSamp, n ,hp , probs, w,wsd,mu_ra, mu_dec,sd_mu_ra, sd_mu_dec,corr_w_mu_ra,corr_w_mu_dec,corr_mu_ra_dec)
    
    print('\033[1m' + 'Velocity estimation:' + '\033[0m')
    
    if isinstance(vRes, str):
        print('')
        print(f"\033[1;31m {vRes} \033[0m")
        print('')
    else:
        v_ra_res, v_dec_res, v_corr_res, Sigma_rv_ra, Sigma_rv_dec,velocitySamples_allr_mean = vRes
        
        #print summary statistics: 
        
        print('')
        print('proper motion ra [mas/yr]:',mu_ra)
        print('proper motion dec [mas/yr]:',mu_dec)
        print('proper motion ra error [mas/yr]:',sd_mu_ra)
        print('proper motion dec error [mas/yr]:',sd_mu_dec)
        print('parallax - proper motion ra correlation: ',corr_w_mu_ra)
        print('parallax - proper motion dec correlation: ',corr_w_mu_dec)
        print('proper motion ra-dec correlation: ',corr_mu_ra_dec)
        print('')
        print('Number of velocity samples drawn for each of the MCMC distance samples: ',n)
        print('')
        print('Estimated velocities:')
        print('')
        print('velocity in ra:')
        print('v_ra_est [km/s] (quantile 0.5): ', v_ra_res[0])
        print('v_ra_lo_est [km/s] (quantile 0.159): ',v_ra_res[1])
        print('v_ra_hi_est [km/s](quantile 0.841): ',v_ra_res[2])
        print('')
        print('velocity in dec:')
        print('v_dec_est [km/s] (quantile 0.5): ', v_dec_res[0])
        print('v_dec_lo_est [km/s] (quantile 0.159): ',v_dec_res[1])
        print('v_dec_hi_est [km/s](quantile 0.841): ',v_dec_res[2])
        print('')
        print('Correlation between v_ra and v_dec: ', v_corr_res[1,0])
        print('')
        print('Covariance between each velocity and distance:' )
        print('Sigma_rv_ra [kpc km/s]: ',Sigma_rv_ra )
        print('Sigma_rv_dec [kpc km/s]: ',Sigma_rv_dec )

/tmp/ipykernel_912276/1704150038.py:35: UserWarning: Warning: converting a masked element to nan.
  pseudocolour = float(r['pseudocolour'])
/jupyter/ad/no233/linux/R/interactive_distance_estimation/functions.py:202: RuntimeWarning: divide by zero encountered in log10
  return np.array([np.log10(ud_distpost4(r=r,w=w,wsd=wsd,rlen=rlen,alpha=alpha,beta=beta)),0],dtype=object)


Gaia DR3 source_id: 3490289711213205632
HEALpixel level 5: 6200
Model of distance prior:  GGD

Distance estimation:

w[mas] 0.5982380400752496
Zeropoint wzp [mas] -0.011402
Zeropointcorrected parallax w[mas] 0.6096400400752496
wsd/w 1.0832977266415558
alpha 0.231898384915402
beta 4.82522701498775
rlen [pc]: 0.00131482073400735

glon [deg] for HEALpixel 6200: 285.73584
glat [deg] for HEALpixel 6200: 34.80791

Estimated distances:

r_est [pc] (quantile 0.5): 1240.6428942244293
r_lo_est [pc] (quantile 0.159): 826.6270042371277
r_hi_est [pc] (quantile 0.841): 2424.0349142787904


MCMC initialization [pc]: 1158.5106340258121
MCMC stepsize [pc]: 289.62765850645303
MCMC number of burn-in samples: 50
MCMC number of retained iterations: 500


Velocity estimation:

proper motion ra [mas/yr]: 3.8943519459022378
proper motion dec [mas/yr]: 0.6259127892169852
proper motion ra error [mas/yr]: 0.6698599457740784
proper motion dec error [mas/yr]: 0.5906122922897339
parallax - proper motion ra correlat